In [1]:
import numpy as np

# Definition of the states, accions y parameters
alpha = 0.3 # 30%
Normal = np.random.normal

# State space
X = [i for i in range (-5, 6)]

# Control space 
U = [0, 1]

# Latest time
T = 3

In [2]:
class Node:
    def __init__(self, t, a, father = None):
        self.t = t # Time
        self.a = a # Action
        self.father = father
        self.w = 0 # cost
        self.x = 0 # state
        self.childs = []
        self.policy = None
    def new_child(self, node):
        self.childs.append(node) # Add the new child


In [3]:
# Define costs
def cost(t, s, x, a):
    if s == 0 and a == 0:
        return 0
    if s == 0 and a == 1:
        return 1
    if s == 1: # Cot of terminal nodes in this case
        return np.exp(-x) - t 


# To pre calculate all terminal nodes and save them
# w_{t,1}
def precalc_terminals_w(X, T):
    w = dict()
    for t in range(T+1):
        s = t
        s_ = dict()
        for u in U:
            s_[u] = dict()
        
        # This is for this case only
        for x in X: 
            s_[1][x] = np.exp(-x) - t # Cost of terminal nodes in this case
        w[s] = s_

    return w

def precal_Q(X):
    Q = dict()
    for x in X:
        Q[x] = dict()
        Q[x][0] = dict()
        Q[x][0][0] = {x-1: 0.2, x-1: 0.2, x: 0.2, x+1: 0.2, x+2: 0.2}
        Q[x][0][1] = {x+1: 0.4, x+2: 0.2, x+3: 0.4}
        
        Q[x][1] = dict()
        Q[x][1][0] = {x-1: 0.6, x: 0.2, x+1: 0.2}
        Q[x][1][1] = {x-1: 0.2, x: 0.4, x+1: 0.4}
    return Q

# In this case we have w_{t, 1} where 1 is the control space chosen
w = precalc_terminals_w(X, T)
# w[t][1]
w[0] = 0


Q_ = precal_Q(X)

In [4]:
def Q(s, x, d): #Q^{t,s}(x, d) # In this case, q does not depend on t
    return Q_[x][s][d]

In [11]:
def AVAR(q, w, alpha):
    # we recive w = w_{t+1}

    # this is something like {w(1): 20%, w(): 60%}
    eval = [(w[max(-5, min(k, 5))], q[k])  for k in q.keys()]
    s = sorted(eval, reverse=True)
    res = 0
    i = 0
    a = alpha
    while(alpha > 0):
        if alpha >= s[i][1]:
            res += s[i][0]*s[i][1]
            alpha -= s[i][1]
        else:
            res += s[i][0]*alpha
            alpha = 0
        i += 1
    return res/a

In [19]:
policy = {}
t = 2
s = 0
while t > 0:
    for x in X:
        if t == 2:
            policy[x] = min(U, key=lambda u: cost(t, s, x, u) + AVAR(Q(s, x, u), w[t+1][1], alpha))
            u = policy[x]
            w[t][s][x] = cost(t, s, x, u) + AVAR(Q(s, x, u), w[t+1][1], alpha)
        else:
            policy[x] = min(U, key=lambda u: cost(t, s, x, u) + AVAR(Q(s, x, u), w[t+1][1], alpha) + AVAR(Q(s, x, u), w[t+1][0], alpha))
            u = policy[x]
            w[t][s][x] = cost(t, s, x, u) + AVAR(Q(s, x, u), w[t+1][1], alpha) + AVAR(Q(s, x, u), w[t+1][0], alpha)

        print(f'w{t, s, x} =, {w[t][s][x]}, policy: {u} ')
    t = t-1

w(2, 0, -5) =, 52.598150033144236, policy: 1 
w(2, 0, -4) =, 18.085536923187668, policy: 1 
w(2, 0, -3) =, 5.38905609893065, policy: 1 
w(2, 0, -2) =, 0.7182818284590451, policy: 1 
w(2, 0, -1) =, -1.0, policy: 1 
w(2, 0, 0) =, -1.6321205588285577, policy: 1 
w(2, 0, 1) =, -2.210706852942853, policy: 0 
w(2, 0, 2) =, -2.7096352781401674, policy: 0 
w(2, 0, 3) =, -2.8931807883863034, policy: 0 
w(2, 0, 4) =, -2.9607034081251795, policy: 0 
w(2, 0, 5) =, -2.9855435917411484, policy: 0 
w(1, 0, -5) =, 71.68368695633191, policy: 1 
w(1, 0, -4) =, 24.47459302211832, policy: 1 
w(1, 0, -3) =, 7.107337927389695, policy: 1 
w(1, 0, -2) =, 0.7182818284590451, policy: 1 
w(1, 0, -1) =, -1.6321205588285577, policy: 1 
w(1, 0, 0) =, -2.8428274117714105, policy: 1 
w(1, 0, 1) =, -3.5742999949035545, policy: 1 
w(1, 0, 2) =, -4.086651606148792, policy: 0 
w(1, 0, 3) =, -4.663997903275183, policy: 0 
w(1, 0, 4) =, -4.876391736424441, policy: 0 
w(1, 0, 5) =, -4.954527061071651, policy: 0 
